In [24]:
# root = '/data-0/qibo/Gdata/col_4/raw'
# x, y = read_cashbus_data(root)

In [ ]:
# get feats; torch;
final2 = pd.read_csv('/home/tigergraph/GraphProject/OneMonthGraph/Querys/pyG/data/feat.csv')
idx2phone = dict(enumerate(final2.v_id))
phone2idx = {v:k for k,v in idx2phone.items()}
x = torch.tensor(final2.iloc[:,1:].values, dtype=torch.float)

# get edges;
edges_df = pd.read_csv('/home/tigergraph/GraphProject/OneMonthGraph/Querys/pyG/data/label.csv')

# get labels；
label_df = pd.read_csv('/home/tigergraph/GraphProject/OneMonthGraph/Querys/pyG/data/label.csv')


In [3]:
import json
import scipy
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy import sparse
import pickle
import torch
from torch_geometric.data import Data

import torch
import torch.nn.functional as F
# from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import SplineConv,GCNConv


import torch
from torch_geometric.data import InMemoryDataset, download_url
# from torch_geometric.read import read_planetoid_data
def sample_mask(index, num_nodes):
    mask = torch.zeros((num_nodes, ), dtype=torch.uint8)
    mask[index] = 1
    return mask

def split_idx(y, split):
    if isinstance(y, torch.Tensor):
        y = torch.tensor(y)
    x_perm = y[torch.randperm(len(y))]
    train_end_idx = int(len(y)*split[0])
    valid_end_idx = int(len(y)*(split[0]+split[1]))
    train_idx = x_perm[:train_end_idx]
    valid_idx = x_perm[train_end_idx:valid_end_idx]
    test_idx = x_perm[valid_end_idx:]
    return train_idx, valid_idx, test_idx

def samples_split_mask(y, split=(0.6,0.2)):
    train_idx, valid_idx, test_idx = split_idx(y, split)
    train_mask = sample_mask(train_idx, num_nodes=y.size(0))
    val_mask = sample_mask(valid_idx, num_nodes=y.size(0))
    test_mask = sample_mask(test_idx, num_nodes=y.size(0))
    return train_mask, val_mask, test_mask

def read_cashbus_data(root):
    # -- userid2idx;
    with open(root + '/col_4_dict', 'r') as f:
        userid2idx = json.load(f)
    # -- node_fetas;  labels; edges;
    x = sparse.load_npz(root + "/feat.npz")
    y = sparse.load_npz(root + "/y.npz")
    edges = sparse.load_npz(root + "/edges.npz").toarray().T
    
    x = torch.tensor(x.toarray(), dtype =torch.float)
    y = torch.tensor(y.toarray()).squeeze()
    row1 = [userid2idx[str(i)] for i in edges[0]]
    row2 = [userid2idx[str(i)] for i in edges[1]]
    new_edges = torch.tensor(np.stack([row1, row2]))
    train_mask, val_mask, test_mask = samples_split_mask(y, split=(0.6,0.2))
    data = Data(x=x, edge_index=new_edges, y=y)
    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask
    return data


class CashBus(InMemoryDataset):
    r"""The citation network datasets "Cora", "CiteSeer" and "PubMed" from the
    `"Revisiting Semi-Supervised Learning with Graph Embeddings"
    <https://arxiv.org/abs/1603.08861>`_ paper.
    Nodes represent documents and edges represent citation links.
    Training, validation and test splits are given by binary masks.
    Args:
        root (string): Root directory where the dataset should be saved.
        name (string): The name of the dataset (:obj:`"Cora"`,
            :obj:`"CiteSeer"`, :obj:`"PubMed"`).
        transform (callable, optional): A function/transform that takes in an
            :obj:`torch_geometric.data.Data` object and returns a transformed
            version. The data object will be transformed before every access.
            (default: :obj:`None`)
        pre_transform (callable, optional): A function/transform that takes in
            an :obj:`torch_geometric.data.Data` object and returns a
            transformed version. The data object will be transformed before
            being saved to disk. (default: :obj:`None`)
    """


    def __init__(self, root, transform=None, pre_transform=None):
        super(CashBus, self).__init__(root, transform, pre_transform)
        print('processed_path:{}'.format(self.processed_paths))
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['col_4_dict', 'feat.npz', 'y.npz', 'edges.npz', 'col_name.pkl']

    @property
    def processed_file_names(self):
        return 'dat.pt'
    
    def download(self):
        pass

    def process(self):
        print('go pl, raw_dir:{}'.format(self.raw_dir))
        data = read_cashbus_data(self.raw_dir)
        data = data if self.pre_transform is None else self.pre_transform(data)
        data, slices = self.collate([data])
        torch.save((data, slices), self.processed_paths[0])

    def __repr__(self):
        return '{}()'.format(self.name)

In [4]:
root = '/data-0/qibo/Gdata/col_4/'
dataset = CashBus(root)
data = dataset[0]


Processing...
go pl, raw_dir:/data-0/qibo/Gdata/col_4/raw
Done!
processed_path:['/data-0/qibo/Gdata/col_4/processed/dat.pt']


/home/qibo/qb_vir_env/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [ ]:
data

In [8]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16, cached=True)
        self.conv2 = GCNConv(16, dataset.num_classes, cached=True)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()

def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 20):
    print(epoch)
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, train_acc, best_val_acc, test_acc))

1
Epoch: 001, Train: 0.5000, Val: 0.5000, Test: 0.5000
2
Epoch: 002, Train: 0.5000, Val: 0.5000, Test: 0.5000
3
Epoch: 003, Train: 0.5000, Val: 0.5000, Test: 0.5000
4
Epoch: 004, Train: 0.5000, Val: 0.5000, Test: 0.5000
5
Epoch: 005, Train: 0.5000, Val: 0.5000, Test: 0.5000
6
Epoch: 006, Train: 0.5000, Val: 0.5000, Test: 0.5000
7
Epoch: 007, Train: 0.5000, Val: 0.5000, Test: 0.5000
8
Epoch: 008, Train: 1.0000, Val: 1.0000, Test: 1.0000
9
Epoch: 009, Train: 1.0000, Val: 1.0000, Test: 1.0000
10
Epoch: 010, Train: 1.0000, Val: 1.0000, Test: 1.0000
11
Epoch: 011, Train: 1.0000, Val: 1.0000, Test: 1.0000
12
Epoch: 012, Train: 1.0000, Val: 1.0000, Test: 1.0000
13
Epoch: 013, Train: 1.0000, Val: 1.0000, Test: 1.0000
14
Epoch: 014, Train: 1.0000, Val: 1.0000, Test: 1.0000
15
Epoch: 015, Train: 1.0000, Val: 1.0000, Test: 1.0000
16
Epoch: 016, Train: 1.0000, Val: 1.0000, Test: 1.0000
17
Epoch: 017, Train: 1.0000, Val: 1.0000, Test: 1.0000
18
Epoch: 018, Train: 1.0000, Val: 1.0000, Test: 1.0000
1